In [22]:
import cv2
import numpy as np
from pyzbar import pyzbar

In [23]:
# Định nghĩa bộ lọc 3x3 trung vị của ảnh
def medianImg3x3(img):
    m, n = img.shape
    img_new = np.zeros_like(img)
    for i in range(1, m-1):
        for j in range(1, n-1):
            temp = [img[i-1, j-1], img[i-1, j], img[i-1, j + 1],
                    img[i, j-1], img[i, j], img[i, j + 1],
                    img[i + 1, j-1], img[i + 1, j], img[i + 1, j + 1]]

            temp = sorted(temp)
            img_new[i, j] = temp[4]     

    return img_new

In [24]:
# Filter làm nét ảnh
def sharpen_image(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    
    sharpened_image = cv2.filter2D(image, -1, kernel)
        
    return sharpened_image

In [25]:
def rotate_qr_code(newImg):    
    gray = cv2.cvtColor(newImg, cv2.COLOR_BGR2GRAY)
    gray = 255 - gray
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    coords = np.column_stack(np.where(thresh > 0))
    angle = cv2.minAreaRect(coords)[-1]

    if angle < -45:
        angle = -(90 + angle)   
    else:
        angle = -angle

    (h, w) = newImg.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(newImg, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

In [26]:
def view(qrCodes_processed, image, img):    
    for qrCode in qrCodes_processed:
        (x, y, w, h) = qrCode.rect
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 255), 2)
        qrCodeData = qrCode.data.decode("utf-8")
        qrCodeType = qrCode.type
        text = "{} - {}".format(qrCodeData, qrCodeType)
        print(text)
        cv2.putText(image, text, (x-10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
       
        qr_code_region = image[y:y+h, x:x+w]

    if img is not None:
         cv2.imshow("Anh goc", img)
         
    cv2.imshow("phat hien", image)

    rotated = rotate_qr_code(qr_code_region)
    cv2.imshow('QR duoc cat ra', rotated)

    key = cv2.waitKey(0)

    if key == ord('q'):
            cv2.destroyAllWindows()

    return qr_code_region

In [27]:
def detect_qr_from_image(image_path):
    image = cv2.imread(image_path)
    imgCopy = cv2.imread(image_path, cv2.IMREAD_COLOR)

    if image is None:
        print("Không thể đọc ảnh từ đường dẫn được cung cấp.")
        return

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    qrCodes = pyzbar.decode(image)

    if len(qrCodes) == 0:
        print("Không tìm thấy mã QR trong ảnh gốc.")

        # Cân bằng sáng
        clahe = cv2.createCLAHE(clipLimit=15.0, tileGridSize=(8, 8))
        equalized_gray = clahe.apply(gray)

        qrCodes = pyzbar.decode(equalized_gray)
        
        if len(qrCodes) == 0:
            print("Không phát hiện được mã QR trên ảnh sau khi được xử lý bằng cân bằng histogram")
            # Xử lý ảnh bằng medianImg3x3
            processed_image = medianImg3x3(gray)
            # Phát hiện mã QR trên ảnh đã được làm mờ
            qrCodes = pyzbar.decode(processed_image)

            if len(qrCodes) == 0:
                print("Không phát hiện được mã QR trên ảnh sau khi được xử lý bằng cân bằng trung vị")
                sharpen = sharpen_image(gray)
                qrCodes = pyzbar.decode(sharpen)
                if len(qrCodes) == 0:
                    print("Không phát hiện được mã QR trên ảnh sau khi qua các bước xử lý ảnh")
                else:
                    print("Phát hiện mã QR trong ảnh đã được làm nét.")
                    sharpen = cv2.cvtColor(sharpen, cv2.COLOR_GRAY2BGR)
                    view(qrCodes, sharpen,imgCopy)
            else:
                print("Phát hiện mã QR trong ảnh đã được xử lý khử nhiễu.")
                processed_image = cv2.cvtColor(processed_image, cv2.COLOR_GRAY2BGR)
                view(qrCodes, processed_image,imgCopy)
                
        else:
            print("Phát hiện mã QR trong ảnh đã được xử lý cân bằng sáng.")
            equalized_gray = cv2.cvtColor(equalized_gray, cv2.COLOR_GRAY2BGR)
            view(qrCodes, equalized_gray, imgCopy)
    else:
        print("Phát hiện mã QR trong ảnh gốc.")
        view(qrCodes, image, None)




In [28]:
detect_qr_from_image("./Img/case10.png")
#  nhấn "q" để tắt cửa sổ cv.imshow
# case 1 ảnh bình thường
# case 4 ảnh nhiễu
# case 5 ảnh tối
# case 8 ảnh mờ
# case 9 ảnh không có gì
# Case 7, 10 ảnh nghiêng

Phát hiện mã QR trong ảnh gốc.
TEAM HOI DAP IS DA BEST! - QRCODE
Skew angle:  -62.10273361206055
